<a href="https://colab.research.google.com/github/Will-Li-zw/Mais202_Assignments/blob/master/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle.json': b'{"username":"liwill","key":"91bd23db2ac34c98468d149b48c02fa3"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c winter2020-mais-202

train_labels.csv: Skipping, found more recently modified local copy (use --force to force download)
train_images.npy.zip: Skipping, found more recently modified local copy (use --force to force download)
test_images.npy.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
label_int_to_str_mapping.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip test_images.npy.zip
!unzip train_images.npy.zip
!ls

Archive:  test_images.npy.zip
replace test_images.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test_images.npy         
Archive:  train_images.npy.zip
replace train_images.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train_images.npy        
'kaggle (1).json'   label_int_to_str_mapping.csv   test_images.npy.zip
'kaggle (2).json'   sample_data			   train_images.npy
'kaggle (3).json'   sample_submission.csv	   train_images.npy.zip
 kaggle.json	    test_images.npy		   train_labels.csv


In [ ]:
!pip install tensorflow==1.14
import tensorflow as tf
import pandas as pd
import numpy as np
import keras

In [ ]:
from keras.models import Sequential #for neural network models
from keras.layers import Dense, Dropout, Flatten, ZeroPadding2D, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator #Data augmentation and preprocessing
from keras.utils import to_categorical #For One-hot Encoding
from keras.optimizers import Adam, SGD, RMSprop #For Optimizing the Neural Network
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

def show_image(arr):
    two_d = (np.reshape(arr, (28, 28)) * 255).astype(np.uint8)
    plt.imshow(two_d, interpolation='nearest')
    plt.show()

X_train = np.load('train_images.npy')[:45000]
y_train = pd.read_csv('train_labels.csv', index_col=0)[:45000]

# for i in range(30,40):
#   show_image(X_train[i])
#   print(y_train[i])

X_test =  np.load('train_images.npy')[45000:50000]
y_test = pd.read_csv('train_labels.csv', index_col=0)[45000:50000]

X_train = X_train/255
X_test =X_test/255
X_train = X_train.reshape(-1,28,28,1)
y_train = keras.utils.to_categorical(np.array(y_train))

X_test = X_test.reshape(-1,28,28,1)
y_test = keras.utils.to_categorical(np.array(y_test))


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import losses
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2


model = Sequential()
model.add(keras.layers.Conv2D(
    filters=32,
    kernel_size=(3,3),
    padding="same",
    activation="relu",
    input_shape = (28,28,1)
))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Conv2D(
    filters=64,
    kernel_size=(3,3),
    padding="same",
    activation="relu"
))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax',kernel_regularizer=l2(0.01)))
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=64,
          epochs=40,
          verbose=1,
          validation_data=(X_test, y_test),callbacks=[learning_rate_reduction])

RuntimeError: ignored

In [ ]:


# test = np.load('test_images.npy')


# test = test.reshape(-1,28,28,1)
# predictions = model.predict(test)
# predictions = np.argmax( predictions ,axis=1)
# predictions = pd.DataFrame(predictions, columns=['label'])
# predictions.index.name = 'ID'
# predictions.to_csv('predictions.csv')

# predictions = model.predict(X_test)
# predictions = np.argmax( predictions ,axis=1)
# predictions = pd.DataFrame(predictions, columns=['label'])
# predictions.index.name = 'ID'
#predictions.to_csv('predictions.csv')

x_test = np.load('test_images.npy')
x_test=x_test/255
x_test = x_test.reshape(-1,28,28,1)
predict = model.predict(x_test,batch_size=256,verbose=1)

predict = np.argmax(predict,axis=1)

df_test = pd.read_csv('sample_submission.csv')
df_test['label'] = predict
df_test.to_csv('submission.csv', index=False)

show_image(x_test[19995])


In [ ]:
show_image(x_test[58])